<h2>Testing and Evaluating</h2>
This create the image and testing the algorithm with numerical and visual result <br>
1. Run time test <br>
2. Performace test <br>
3. Kernel's size and weight VS noise <br>

In [ ]:
import numpy as np
from PIL import Image, ImageFilter, ImageOps
from ImageFrame import Frame
from EdgeFinderV4 import BoundaryDetector as BD2
import timeit
import cv2 as cv
import matplotlib.pylab as plt

In [ ]:
frame1 = Frame(1280,720,5,100,600)
kernal = np.array([-1,-1,-1,0,1,1,1])
T_hold = 75

In [ ]:
plt.axis((0,1280,720,0))
for key in frame1.fline.keys():
    plt.plot(frame1.fline[key][:,0],frame1.fline[key][:,1],'k,')
plt.show()

Run time testing with openCV and our Modified lane-markers detection

In [ ]:
#compiling the function
img = np.array(Image.open("TestingImage/Image"+str(1)+".jpg"))[:,:,0]
BD2(img,kernal,frame1,T_hold) 


In [ ]:
m1_time = np.empty([0,0])
m2_time = np.empty([0,0])
m3_time = np.empty([0,0])
ddepth = cv.CV_16S

for i in range(50):
    img = np.array(Image.open("TestingImage/Image"+str(i)+".jpg"))[:,:,0]
    img1 = cv.imread("TestingImage/Image"+str(i)+".jpg")
  

    # Pre-process for Sobel algorithm

    t1 = timeit.default_timer()
    # Modified Edge detection algorithm
    
    B = BD2(img,kernal,frame1,T_hold) 

    t2 = timeit.default_timer()
    # Canny operator
    c=cv.Canny(img1,100,200)
    
    t3 = timeit.default_timer()

    # Sobel Operator
    img3 = cv.GaussianBlur(img1,(3,3),0)
    img2 = cv.cvtColor(img3,cv.COLOR_BGR2GRAY)
    grad_x = cv.Sobel(img2, ddepth, 1, 0, ksize=3, scale=1, delta=0, borderType=cv.BORDER_DEFAULT)
    grad_y = cv.Sobel(img2, ddepth, 0, 1, ksize=3, scale=1, delta=0, borderType=cv.BORDER_DEFAULT)
    abs_grad_x = cv.convertScaleAbs(grad_x)
    abs_grad_y = cv.convertScaleAbs(grad_y)
    grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    t4 = timeit.default_timer()

    
    #B.ploting_onImage(img,str(i)+" \n our_method:"+str(np.round(t2-t1,4))+"s\n Canny:"+str(np.round(t3-t2,4))+"s")
    #plt.show()


    m1_time = np.append(m1_time,t2-t1)
    m2_time = np.append(m2_time,t3-t2)
    m3_time = np.append(m3_time,t4-t3)


#printing statistic for both method
print(f'Our_method : max={np.max(m1_time): .4f} min={np.min(m1_time): .4f} average={np.average(m1_time): .4f}')
print(f'Canny_method : max={np.max(m2_time): .4f} min={np.min(m2_time): .4f} average={np.average(m2_time): .4f}')
print(f'Sobel_method : max={np.max(m3_time): .4f} min={np.min(m3_time): .4f} average={np.average(m3_time): .4f}')

<h2>Performance testing</h2>


In [ ]:
frame1 = Frame(1280,720,5,720/2,720)
fig, ax = plt.subplots(8,4,figsize=(19,25))
for n in range(1,33,1):
    
    img = np.array(Image.open("TestingImage/Image"+str(n+32+4)+".jpg"))
    #applying the masks to the image
    kernal = np.array([-1,-1,-1,0,1,1,1])
    #Using EdgeDetection
    BP = BD2(img[:,:,2],kernal,frame1,60)

    P = BP.boundary
    
    #print(P)
    #A = get_angle(P)    
    ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].imshow(img)
    for key in frame1.fline.keys():
        ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].plot(frame1.fline[key][:,0],frame1.fline[key][:,1],'k,')
    ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].set_title("Image {num}".format(num=n+32+4))
    ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].plot(BP.boundary[:,0],BP.boundary[:,1],"ro")
    ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].set_xticks([]), ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].set_yticks([])

    #BP.ploting_onImage(img,str(n)+' case '+str(A[0]))
    #plt.plot(A[2][:,0], 720-A[2][:,1], 'b.')
    #plt.plot([1280/2,1280/2+200*np.sin(np.deg2rad(A[1][0]))],[720-0.0,720-200*np.cos(np.deg2rad(A[1][0]))],'b-')
    #print(A)        
    # 
plt.show()

In [ ]:
frame1 = Frame(1280,720,5,100,600)
fig, ax = plt.subplots(3,4,figsize=(19,7))
for n in range(1,13,1):
    
    img = Image.open("CroppedImage/photo"+str(n)+".jpg")
    #img = np.array(ImageOps.grayscale(img).filter(ImageFilter.GaussianBlur(radius=2)))
    img = np.array(img)[:,:,2]
    #applying the masks to the image
    kernal = np.array([-1,-2,0,2,1])
    #Using EdgeDetection
    BP = BD2(img,kernal,frame1,150)

    P = BP.boundary
    
    #print(P)
    #A = get_angle(P)    
    ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].imshow(img,cmap='gray')
    for key in frame1.fline.keys():
        ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].plot(frame1.fline[key][:,0],frame1.fline[key][:,1],'k,')
    ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].set_title("Image {num}".format(num=n))
    ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].plot(BP.boundary[:,0],BP.boundary[:,1],"ro")
    ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].set_xticks([]), ax[int((n-1)/4),(n-1)-4*int((n-1)/4)].set_yticks([])

    #BP.ploting_onImage(img,str(n)+' case '+str(A[0]))
    #plt.plot(A[2][:,0], 720-A[2][:,1], 'b.')
    #plt.plot([1280/2,1280/2+200*np.sin(np.deg2rad(A[1][0]))],[720-0.0,720-200*np.cos(np.deg2rad(A[1][0]))],'b-')
    #print(A)        
    # 
plt.show()

In [ ]:
img = np.array(Image.open("TestingImage/Image32.jpg"))[:,:,0]

bP = BD2(img,kernal,frame1,60)
data = np.empty([int(frame1.width/2.0),0],int)
for key in frame1.fline.keys():
    data = np.append(data,np.array([[img[int(y),int(x)]] for x,y in frame1.fline[key]]),axis=1)
fig,axs=plt.subplots(6,3,figsize=(20,30))

for i,j in enumerate(bP.boundary[:,0]):
    if i%2:
        axs[int(i/3),i-3*int(i/3)].set_title(f'line: left {i+1}')
    else:
        axs[int(i/3),i-3*int(i/3)].set_title(f'line: right {i+1}')
    axs[int(i/3),i-3*int(i/3)].axis((0,640,0,140))
    axs[int(i/3),i-3*int(i/3)].plot(range(0,data.shape[0]),data[:,i],'k.')
    
    axs[int(i/3),i-3*int(i/3)].plot(range(int((len(kernal))/2)+20,bP.conv_resutl.shape[1]+int((len(kernal))/2)+20),bP.conv_resutl[i,:],'b-')
    
    if not j:
        continue
    if not i%2:
        axs[int(i/3),i-3*int(i/3)].plot([j-data.shape[0],j-data.shape[0]],[100,160],'r-')
    else:
        axs[int(i/3),i-3*int(i/3)].plot([data.shape[0]-j,data.shape[0]-j],[100,160],'r-')
plt.show()

In [ ]:
img = Image.open("CroppedImage/photo1.jpg")
#img = np.array(ImageOps.grayscale(img).filter(ImageFilter.GaussianBlur(radius=2)))
img = np.array(img)[:,:,0]
kernal = np.array([-1,-2,4/16,2,1])

bP = BD2(img,kernal,frame1,150)
data = np.empty([int(frame1.width/2.0),0],int)
for key in frame1.fline.keys():
    data = np.append(data,np.array([[img[int(y),int(x)]] for x,y in frame1.fline[key]]),axis=1)
fig,axs=plt.subplots(6,3,figsize=(20,30))

for i,j in enumerate(bP.boundary[:,0]):
    if i%2:
        axs[int(i/3),i-3*int(i/3)].set_title(f'line: left {i+1}')
    else:
        axs[int(i/3),i-3*int(i/3)].set_title(f'line: right {i+1}')
    axs[int(i/3),i-3*int(i/3)].axis((0,640,0,250))
    axs[int(i/3),i-3*int(i/3)].plot(range(0,data.shape[0]),data[:,i],'k.')
    
    axs[int(i/3),i-3*int(i/3)].plot(range(int((len(kernal))/2)+20,bP.conv_resutl.shape[1]+int((len(kernal))/2)+20),bP.conv_resutl[i,:],'b-')
    
    if not j:
        continue
    if not i%2:
        axs[int(i/3),i-3*int(i/3)].plot([j-data.shape[0],j-data.shape[0]],[100,160],'r-')
    else:
        axs[int(i/3),i-3*int(i/3)].plot([data.shape[0]-j,data.shape[0]-j],[100,160],'r-')
plt.show()

Noise Test with different size and weight kernel

In [ ]:
img = np.array(Image.open("TestingImage/Image18.jpg"))[:,:,0]
frame1 = Frame(1280,720,10,720/2,720)
data = np.empty([int(frame1.width/2.0),0],int)
#img[int(frame1.fline['4'][-500][1]),499]=200  #injecting noise to the data

for key in frame1.fline.keys():
    data = np.append(data,np.array([[img[int(y),int(x)]] for x,y in frame1.fline[key]]),axis=1)
fig,axs=plt.subplots(2,2,figsize=(22,10))

def plotMe(i,n,line,bP,kernal,title,mode=0):
    axs[i,n].set_title(f'{title}')
    if mode:
        axs[i,n].axis((0,640,0,1400))
    else:
        axs[i,n].axis((0,640,0,300))

    axs[i,n].plot(range(0,data.shape[0]),data[:,line],'k.')

    axs[i,n].plot(range(int((len(kernal))/2)+20-3,bP.conv_resutl.shape[1]+int((len(kernal))/2)+20-3),bP.conv_resutl[line,:],'b-')

        
kernel = np.array([-1,0,1])
bP = BD2(img,kernel,frame1,500)
plotMe(0,0,3,BD2(img,kernel,frame1,500),kernel,'Kernel [-1,0,1]')
kernel = np.array([-1,-1,0,1,1])
plotMe(0,1,3,BD2(img,kernel,frame1,500),kernel,'Kernel [-1,-1,0,1,1]')
kernel = np.array([-1,-1,-1,0,1,1,1])
plotMe(1,0,3,BD2(img,kernel,frame1,500),kernel,'Kernel [-1,-1,-1,0,1,1,1]')
kernel = np.array([-1,-2,0,2,1])
plotMe(1,1,3,BD2(img,kernel,frame1,500),kernel,'Kernel [-1,-2,0,2,1]')

plt.show()


In [ ]:
img = Image.open("CroppedImage/photo1.jpg")
img = np.array(ImageOps.grayscale(img).filter(ImageFilter.GaussianBlur(radius=2)))
img = np.array(img)
frame1 = Frame(1280,720,5,100,600)
data = np.empty([int(frame1.width/2.0),0],int)
#img[int(frame1.fline['14'][-500][1]),499]=200  #injecting noise to the data

for key in frame1.fline.keys():
    data = np.append(data,np.array([[img[int(y),int(x)]] for x,y in frame1.fline[key]]),axis=1)
fig,axs=plt.subplots(2,2,figsize=(22,10))

def plotMe(i,n,line,bP,kernal,title,mode=0):
    axs[i,n].set_title(f'{title}')
    if mode:
        axs[i,n].axis((0,640,0,1400))
    else:
        axs[i,n].axis((0,640,0,300))

    axs[i,n].plot(range(0,data.shape[0]),data[:,line],'k.')

    axs[i,n].plot(range(int((len(kernal))/2)+20-3,bP.conv_resutl.shape[1]+int((len(kernal))/2)+20-3),bP.conv_resutl[line,:],'b-')

        
kernel = np.array([-1,0,1])
bP = BD2(img,kernel,frame1,500)
plotMe(0,0,13,BD2(img,kernel,frame1,500),kernel,'Kernel [-1,0,1]')
kernel = np.array([-1,-1,0,1,1])
plotMe(0,1,13,BD2(img,kernel,frame1,500),kernel,'Kernel [-1,-1,0,1,1]')
kernel = np.array([-1,-1,-1,0,1,1,1])
plotMe(1,0,13,BD2(img,kernel,frame1,500),kernel,'Kernel [-1,-1,-1,0,1,1,1]')
kernel = np.array([-1,-2,0,2,1])
plotMe(1,1,13,BD2(img,kernel,frame1,500),kernel,'Kernel [-1,-2,0,2,1]')

plt.show()
